In [ ]:
from high_low_xuejieZuhui import *
from horizontal_area import *
import os
from test import *


os.chdir('/Users/kai/Desktop/qs/data_csv_distinct_0606')
for filename in os.listdir():
    # filename = '000001.SZ.csv'
    os.chdir('/Users/kai/Desktop/qs/data_csv_distinct_0606/_horizontal_area_pics')
    df = df_init(filename)
    peaks, valleys, high_points, low_points = find_high_low_xuejieZuhui(
        df, filename, save_data=False, draw_n_days=200, draw=False)

    max_len_of_window = 40
    min_len_of_window = 12
    gamma = 0.8  # price_change_50 >= gamma * max_change:
    view_coe = 0.8  # 视野·系数

    result = find_horizontal_area1(df, high_points, low_points, max_len_of_window,
                                min_len_of_window, gamma, view_coe, ignore_hl=True)

    n_days = 300
    draw_horizontal_area(df, result, peaks, valleys, high_points,
                        low_points, filename, n_days, print_result=False, show_plot=False)
                        
    os.chdir('/Users/kai/Desktop/qs/data_csv_distinct_0606')


In [ ]:
from high_low_xuejieZuhui import *
from horizontal_area import *
import os
from test import *

In [ ]:
# filename = '688699.SH.csv'
filename = '000001.SZ.csv'
df = df_init(filename)
peaks, valleys, high_points, low_points = find_high_low_xuejieZuhui(
    df, filename, save_data=False, draw_n_days=200, draw=False)

max_len_of_window = 40
min_len_of_window = 10
gamma = 0.7  # price_change_50 >= gamma * max_change:
view_coe = 0  # 视野·系数
var = 30

result = find_horizontal_area1(df, high_points, low_points, var, max_len_of_window,
                            min_len_of_window, gamma, view_coe, ignore_hl=True)

n_days = 1000
draw_horizontal_area(df, result, peaks, valleys, high_points,
                    low_points, filename, n_days, print_result=True, show_plot=True)

In [ ]:
n_days = 400
draw_horizontal_area(df, result, peaks, valleys, high_points,
                    low_points, filename, n_days, print_result=False, show_plot=True)

In [5]:
import pandas as pd
df = pd.read_csv('/Users/kai/Desktop/qs/data_csv_distinct_0606/_horizontal_area_pics/results.csv')

In [29]:
df.columns = ['min_len_of_window', 'max_len_of_window', 'n', 'mean']
df.dropna(inplace=True)
for mins in (6,12,18):
    for maxs in (20,30,40,50,60,70):
        temp = df[(df['min_len_of_window']==mins) & (df['max_len_of_window']==maxs)]
        print(mins, maxs, round(temp['mean'].dot(temp['n'])/temp['n'].sum(), 2))

6 20 15.6
6 30 17.35
6 40 18.6
6 50 19.46
6 60 20.37
6 70 21.15
12 20 23.67
12 30 27.09
12 40 29.26
12 50 30.66
12 60 31.65
12 70 32.59
18 20 29.36
18 30 34.92
18 40 38.83
18 50 41.7
18 60 43.36
18 70 44.91


In [21]:
tt = df.iloc[1:3,:]
tt['n'].dot(tt['mean'])/tt['n'].sum()


26.5025